# NLP Project: Scraping GitHub READMEs

In [1]:
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup

import acquire

import unicodedata
import re
import json

import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords

import acquire
import prepare as p

### Wrangle Functions
- Make sure they work

In [2]:
string = ['Hello welcome to Legoland',
          'He is from China but prefers Mexican enchiladas',
          "Peabo hasn't taken a bath in 10 days",
          "I don't recall ever having gone to Europe",
         "Potato thy fine fruit! You are quite the delicious"]

In [3]:
df = pd.DataFrame(data=string, columns=['string'])
df

,string
0,Hello welcome to Legoland
1,He is from China but prefers Mexican enchiladas
2,Peabo hasn't taken a bath in 10 days
3,I don't recall ever having gone to Europe
4,Potato thy fine fruit! You are quite the delic...


In [4]:
p.prepare_nlp_data(df, content = 'string', extra_words=[], exclude_words=[])

,string,clean,stemmed,lemmatized
0,Hello welcome to Legoland,hello welcome legoland,hello welcom legoland,hello welcome legoland
1,He is from China but prefers Mexican enchiladas,china prefers mexican enchiladas,china prefer mexican enchilada,china prefers mexican enchilada
2,Peabo hasn't taken a bath in 10 days,peabo hasnt taken bath 10 days,peabo hasnt taken bath 10 day,peabo hasnt taken bath 10 day
3,I don't recall ever having gone to Europe,dont recall ever gone europe,dont recal ever gone europ,dont recall ever gone europe
4,Potato thy fine fruit! You are quite the delic...,potato thy fine fruit quite delicious,potato thi fine fruit quit delici,potato thy fine fruit quite delicious


In [5]:
# bring it scraped json file
df = pd.read_json('data2.json')

In [6]:
# some chinese repos we need to remove
df.head()

,repo,language,readme_contents
0,codingXiaxw/CustomerManagement,Java,## JavaWeb项目之客户管理系统 \n\n## 项目介绍\n详细讲解请点击这里前往我的...
1,DanWahlin/CustomerManager,JavaScript,Customer Manager with AngularJS (with custom r...
2,Tophold/FinancialCustomerView,Java,## FinancialCustomerView\n* 本项目会对金融交易软件中存在的各种V...
3,DanWahlin/CustomerManagerStandard,JavaScript,Customer Manager with AngularJS\r\n===========...
4,spring-cloud-samples/customers-stores,Java,# REST based micro-services sample\n\n- Three ...


In [7]:
df.shape

(194, 3)

In [8]:
# do drop unneeded data first then prep
df = p.drop_unneeded_data(df)

In [9]:
df.head()

,repo,language,readme_contents
0,DanWahlin/CustomerManager,JavaScript,Customer Manager with AngularJS (with custom r...
1,DanWahlin/CustomerManagerStandard,JavaScript,Customer Manager with AngularJS\r\n===========...
2,spring-cloud-samples/customers-stores,Java,# REST based micro-services sample\n\n- Three ...
3,eventuate-tram/eventuate-tram-examples-custome...,Java,# Eventuate Tram Customers and Orders\n\nThis ...
4,Featuretools/predict-customer-churn,Jupyter Notebook,# A Machine Learning Framework with an Applica...


In [13]:
df = p.prepare_nlp_data(df, content = 'readme_contents', extra_words=['customer', 'customers'], exclude_words=[])

In [14]:
df.head()

,repo,language,readme_contents,clean,stemmed,lemmatized
0,DanWahlin/CustomerManager,JavaScript,Customer Manager with AngularJS (with custom r...,manager angularjs custom routing dynamic contr...,manag angularj custom rout dynam control scrip...,manager angularjs custom routing dynamic contr...
1,DanWahlin/CustomerManagerStandard,JavaScript,Customer Manager with AngularJS\r\n===========...,manager angularjs youre new angularjs check an...,manag angularj your new angularj check angular...,manager angularjs youre new angularjs check an...
2,spring-cloud-samples/customers-stores,Java,# REST based micro-services sample\n\n- Three ...,rest based microservices sample three spring b...,rest base microservic sampl three spring boot ...,rest based microservices sample three spring b...
3,eventuate-tram/eventuate-tram-examples-custome...,Java,# Eventuate Tram Customers and Orders\n\nThis ...,eventuate tram orders application demonstrates...,eventu tram order applic demonstr two key patt...,eventuate tram order application demonstrates ...
4,Featuretools/predict-customer-churn,Jupyter Notebook,# A Machine Learning Framework with an Applica...,machine learning framework application predict...,machin learn framework applic predict churn im...,machine learning framework application predict...
